In [1]:
class BasicBlock:
    def __init__(self, name):
        self.name = name
        self.statements = []
        self.predecessors = []
        self.successors = []


class Statement:
    def __init__(self, variable, expression):
        self.variable = variable
        self.expression = expression

    def is_assignment(self):
        return isinstance(self.expression, AssignmentExpression)


class AssignmentExpression:
    def __init__(self, variable, value):
        self.variable = variable
        self.value = value


def create_sample_cfg():
    # Create basic blocks
    block1 = BasicBlock("Block 1")
    block2 = BasicBlock("Block 2")
    block3 = BasicBlock("Block 3")
    block4 = BasicBlock("Block 4")
    block5 = BasicBlock("Block 5")
    block6 = BasicBlock("Block 6")

    # Create statements
    stmt1 = Statement("a", AssignmentExpression("a", "1"))
    stmt2 = Statement("b", AssignmentExpression("b", "2"))
    stmt3 = Statement("c", AssignmentExpression("c", "a + b"))
    stmt4 = Statement("d", AssignmentExpression("d", "c - a"))
    stmt5 = Statement("d", AssignmentExpression("d", "b + d"))
    stmt6 = Statement("d", AssignmentExpression("d", "a + b"))
    stmt7 = Statement("e", AssignmentExpression("e", "e + 1"))
    stmt8 = Statement("b", AssignmentExpression("b", "a + b"))
    stmt9 = Statement("e", AssignmentExpression("e", "c - a"))
    stmt10 = Statement("a", AssignmentExpression("a", "b * d"))
    stmt11 = Statement("b", AssignmentExpression("b", "a - d"))

    # Connect basic blocks
    block1.successors = [block2]
    
    block2.predecessors = [block1, block5]
    block2.successors = [block3]

    block3.predecessors = [block2, block4]
    block3.successors = [block4, block5]

    block4.predecessors = [block3]
    block4.successors = [block3]

    block5.predecessors = [block3]
    block5.successors = [block2]

    block6.predecessors = [block5]
    


    # Assign statements to basic blocks
    block1.statements = [stmt1, stmt2]
    block2.statements = [stmt3, stmt4]
    block3.statements = [stmt5]
    block4.statements = [stmt6, stmt7]
    block5.statements = [stmt8, stmt9]
    block6.statements = [stmt10, stmt11]


    # Return the CFG
    return [block1, block2, block3, block4, block5, block6]

In [5]:
cfg = create_sample_cfg()
print(cfg[1])

[<__main__.BasicBlock object at 0x111a425d0>, <__main__.BasicBlock object at 0x111a50b90>]


In [9]:
def calculate_reaching_definitions(cfg):
    # Initialize Kill, Gen, In, and Out sets for each basic block
    kill = {block: set() for block in cfg}
    gen = {block: set() for block in cfg}
    in_set = {block: set() for block in cfg}
    out_set = {block: set() for block in cfg}
    print("working....")
    # Calculate Kill and Gen sets for each basic block
    for block in cfg:
        kill[block] = set()  # Initialize Kill set to an empty set

        for stmt in block.statements:
            if stmt.is_assignment():  # Check if statement is an assignment
                kill[block].add(stmt.variable)  # Add assigned variable to Kill set
                gen[block].add(stmt.variable)  # Add assigned variable to Gen set

    # Perform the reaching definitions analysis using a worklist algorithm
    worklist = list(cfg)  # Initialize worklist with all basic blocks
    print("worklist len: ", len(worklist))
    while worklist:
        current_block = worklist.pop(0)

        # Calculate In set for the current block
        in_set[current_block] = set().union(*[out_set[predecessor] for predecessor in current_block.predecessors])


        # Calculate Out set for the current block
        out_set[current_block] = gen[current_block].union(in_set[current_block].difference(kill[current_block]))


        # Update In set for successors of the current block
        print("worklist before")
        for successor in current_block.successors:
            print("worklist AFTER")
            if successor not in worklist:
                worklist.append(successor)

    # Return the calculated sets
    return kill, gen, in_set, out_set

# Assuming you have created the cfg using create_sample_cfg function
cfg = create_sample_cfg()
print("cfg created...")

# Call the function to calculate reaching definitions sets
kill_set, gen_set, in_set, out_set = calculate_reaching_definitions(cfg)

# Now you can inspect the results
print("Kill set:")
for block, variables in kill_set.items():
    print(f"{block.name}: {variables}")

print("\nGen set:")
for block, variables in gen_set.items():
    print(f"{block.name}: {variables}")

print("\nIn set:")
for block, variables in in_set.items():
    print(f"{block.name}: {variables}")

print("\nOut set:")
for block, variables in out_set.items():
    print(f"{block.name}: {variables}")


cfg created...
working....
worklist len:  6


KeyboardInterrupt: 